In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ESRNN import ESRNN
plt.style.use('ggplot')
pd.options.display.max_rows = 999
np.set_printoptions(threshold=np.inf)

In [ ]:
# Plot
def plot_prediction(y, y_hat):
    n_y = len(y)
    n_yhat = len(y_hat)
    ds_y = np.array(range(n_y))
    ds_yhat = np.array(range(n_y-n_yhat, n_y))

    plt.plot(ds_y, y, label = 'y')
    plt.plot(ds_yhat, y_hat, label='y_hat')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
# Plot
def plot_decomposition(trend, seasonality, level):
    ds_y = np.array(range(len(trend)))
    plt.figure(figsize=(10,2))
    plt.plot(ds_y, trend, label = 'trend')
    plt.legend(loc='upper left')
    plt.show()
    
    plt.figure(figsize=(10,2))
    plt.plot(ds_y, seasonality, label='seasonality')
    plt.legend(loc='upper left')
    plt.show()
    
    plt.figure(figsize=(10,2))
    plt.plot(ds_y, level, label='level')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
def ffill_missing_dates_particular_serie(serie, min_date, max_date, freq):
    date_range = pd.date_range(start=min_date, end=max_date, freq=freq)
    unique_id = serie['unique_id'].unique()
    df_balanced = pd.DataFrame({'ds':date_range, 'key':[1]*len(date_range), 'unique_id': unique_id[0]})

    # Check balance
    check_balance = df_balanced.groupby(['unique_id']).size().reset_index(name='count')
    assert len(set(check_balance['count'].values)) <= 1
    df_balanced = df_balanced.merge(serie, how="left", on=['unique_id', 'ds'])

    df_balanced['y'] = df_balanced['y'].fillna(method='ffill')

    return df_balanced

def ffill_missing_dates_per_serie(df, freq, fixed_max_date=None):
    """Receives a DataFrame with a date column and forward fills the missing gaps in dates, not filling dates before
    the first appearance of a unique key

    Parameters
    ----------
    df: DataFrame
        Input DataFrame
    key: str or list
        Name(s) of the column(s) which make a unique time series
    date_col: str
        Name of the column that contains the time column
    freq: str
        Pandas time frequency standard strings, like "W-THU" or "D" or "M"
    numeric_to_fill: str or list
        Name(s) of the columns with numeric values to fill "fill_value" with
    """
    if fixed_max_date is None:
        df_max_min_dates = df[['unique_id', 'ds']].groupby('unique_id').agg(['min', 'max']).reset_index()
    else:
        df_max_min_dates = df[['unique_id', 'ds']].groupby('unique_id').agg(['min']).reset_index()
        df_max_min_dates['max'] = fixed_max_date

    df_max_min_dates.columns = df_max_min_dates.columns.droplevel()
    df_max_min_dates.columns = ['unique_id', 'min_date', 'max_date']

    df_list = []
    for index, row in df_max_min_dates.iterrows():
        df_id = df[df['unique_id'] == row['unique_id']]
        df_id = ffill_missing_dates_particular_serie(df_id, row['min_date'], row['max_date'], freq)
        df_list.append(df_id)

    df_dates = pd.concat(df_list).reset_index(drop=True).drop('key', axis=1)[['unique_id', 'ds', 'y']]

    return df_dates

# Train

In [ ]:
data = pd.read_csv('data/stock/train.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['year'] = data['Date'].dt.year.astype(str)
data['unique_id'] = data['Company']+"_"+data['year']
data = data.rename(columns={'Date':'ds', 'Close':'y'})

#Series must be complete in the frequency
data = ffill_missing_dates_per_serie(data,'D')
data = data.drop_duplicates(['unique_id','ds'])

X_train = data[['unique_id','ds']]
X_train['x'] = '1'
y_train = data[['unique_id','ds','y']]

# Test

In [ ]:
data_test = pd.read_csv('data/stock/test.csv')
data_test['ds'] = pd.to_datetime(data_test['Date'])
data_test['year'] = data_test['ds'].dt.year.astype(str)
data_test['unique_id'] = data_test['Company']+"_"+data_test['year']
X_test = data_test[['unique_id','ds','Close']]
X_test.columns = ['unique_id', 'ds', 'y']
uniques = X_test['unique_id'].unique()

In [ ]:
# Train model
esrnn = ESRNN(max_epochs=100, batch_size=16, learning_rate=3e-4, gradient_clipping_threshold=20,
              dilations=[[1, 7], [28]], add_nl_layer=True, per_series_lr_multip=1.5, 
              seasonality=7, input_size=7, output_size=30, max_periods=12, level_variability_penalty=100)
esrnn.fit(X_train, y_train, random_seed=1)

In [ ]:
y_hat = esrnn.predict(X_test, decomposition=True)
X_plot = y_train.append(y_hat)

In [ ]:
plot_id = 3
y_test_plot = X_plot.loc[X_plot['unique_id']==uniques[plot_id]]
plot_prediction(y_test_plot['y'], y_test_plot['y_hat'])

In [ ]:
plot_decomposition(y_test_plot['trend'], y_test_plot['seasonalities'], y_test_plot['level'])

In [ ]:
np.abs(y_hat['y_hat']-y_hat['y']).mean()